In [1]:
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from typing import Protocol


## Entry Step

In [28]:
# input fir file dir
ic50_path = pathlib.Path(".")
ic50_filename = "FIR_file_for_EC50.xlsx"

ic50_df = pd.read_excel(ic50_path / ic50_filename)

In [3]:
rename_columns = ["EOS", "CONCENTRATION", "VALUE"]
rename_dict = dict(zip(ic50_df.columns, rename_columns))
ic50_df.rename(columns=rename_dict, inplace=True)

In [4]:
ic50_df.sort_values(by=["EOS", "CONCENTRATION"], inplace=True)

## DRC Fitting Step

In [5]:
LOWER_BOUND = -100

inh_valid = ic50_df[ic50_df["VALUE"] >= LOWER_BOUND]

# repeated sorting is redundant

In [6]:
from scipy.optimize import curve_fit

# To determine ec50 or ic50???
def four_param_logistic(x, lower_limit, upper_limit, ec50, slope):
    return lower_limit + (upper_limit - lower_limit) / (1 + (x / ec50) ** slope)

# upper branch does not group cmpds??

by_eos = inh_valid.groupby("EOS")


In [7]:
def plot_curve_fit(x, y, eos, ax):
    popt, _ = curve_fit(four_param_logistic, x, y)
    ax.plot(x, y, 'o', label='data')
    extended_x = np.linspace(min(x), max(x), 100)
    ax.plot(extended_x, four_param_logistic(extended_x, *popt), 'r-', label='fit')
    ax.set_title(f"EOS: {eos}")
    ax.legend(loc='best')
    return popt

In [8]:
fit_props = ["lower_limit", "upper_limit", "ec50", "slope"]
concentration_props = ["min_concentration", "max_concentration"]
curve_fit_params = {key: [] for key in ["EOS", *fit_props, *concentration_props]}
for key, group in by_eos:
    by_conc = group.groupby("CONCENTRATION")
    values_avg = by_conc["VALUE"].mean()
    x = values_avg.index.to_numpy()
    y = values_avg.values
    try:
        params, _ = curve_fit(four_param_logistic, x, y, maxfev=10000)
    except RuntimeError:
        print(f"EOS: {key} - curve_fit failed")
        params = [np.nan] * 4

    curve_fit_params["EOS"].append(key)
    for i, name in enumerate(fit_props):
        curve_fit_params[name].append(params[i])

    curve_fit_params["min_concentration"].append(x.min())
    curve_fit_params["max_concentration"].append(x.max())
    

curve_fit_df = pd.DataFrame(curve_fit_params)

C:\Users\sbart\AppData\Local\Temp\ipykernel_11132\2116374892.py:4: RuntimeWarning: invalid value encountered in power
  return lower_limit + (upper_limit - lower_limit) / (1 + (x / ec50) ** slope)


EOS: EOS101302 - curve_fit failed
EOS: EOS2452 - curve_fit failed
EOS: EOS60470 - curve_fit failed
EOS: EOS84313 - curve_fit failed
EOS: EOS98641 - curve_fit failed


In [9]:
# what is justification for this step?
KNIME_SLOPE_MULTIPLIER = -1.2

curve_fit_df["slope_adjusted"] = round(curve_fit_df.slope * KNIME_SLOPE_MULTIPLIER)

In [10]:
curve_fit_df

,EOS,lower_limit,upper_limit,ec50,slope,min_concentration,max_concentration,slope_adjusted
0,EOS100028,101.686898,-11.652444,5.073602e-01,0.874958,0.0125,50.0,-1.0
1,EOS100057,104.079979,0.546953,2.018330e+00,0.955924,0.0125,50.0,-1.0
2,EOS100080,-47.887953,-0.091180,4.980150e+00,14.636810,0.0125,5.0,-18.0
3,EOS100134,99.885443,-2.483754,8.876217e-01,1.232868,0.0125,50.0,-1.0
4,EOS100147,99.465109,-4.950270,3.961385e-01,1.164803,0.0125,50.0,-1.0
...,...,...,...,...,...,...,...,...
504,EOS98635,100.659103,-14.101436,3.775230e-01,0.898533,0.0125,50.0,-1.0
505,EOS98640,99.999461,-152.610628,3.282508e-03,0.957010,0.0125,50.0,-1.0
506,EOS98641,NaN,NaN,NaN,NaN,0.0125,50.0,NaN
507,EOS98642,100.019671,-640225.681281,5.762685e-08,0.932795,0.0125,50.0,-1.0


In [11]:
class FitDFEntry(Protocol):
    ec50: float
    max_concentration: float
    min_concentration: float
    lower_limit: float
    upper_limit: float
    slope: float
    ec50: float

def resolve_sign(row: FitDFEntry) -> str:
    if row.ec50 > row.max_concentration:
        return '>'
    if row.ec50 < row.min_concentration:
        return '<'
    return '='

curve_fit_df["sign"] = curve_fit_df.apply(resolve_sign, axis=1)

In [12]:
def bounded_ec50(row: FitDFEntry) -> float:
    return np.clip(row.ec50, row.min_concentration, row.max_concentration)


curve_fit_df["bounded_ec50"] = curve_fit_df.apply(bounded_ec50, axis=1)

In [13]:
curve_fit_df_indexed_by_eos = curve_fit_df.set_index("EOS")

## Activity Determination

In [14]:
concentration_grouped_df = (
    ic50_df
    .groupby(["EOS", "CONCENTRATION"])
    .VALUE
    .aggregate(["mean", "min", "max"])
    .reset_index()
)


In [15]:
# sorter branch groupped by CMPD ID and concentration, DRC only by CMPD ID
activation_df = (
    concentration_grouped_df
    .merge(
        curve_fit_df_indexed_by_eos, 
        how="inner", 
        left_on="EOS", 
        right_on="EOS"
    )
    .rename(
        columns={
            "min": "min_value",
            "max": "max_value",
            "mean": "mean_value"
        }
    )
)

In [16]:
activation_df.head()

,EOS,CONCENTRATION,mean_value,min_value,max_value,lower_limit,upper_limit,ec50,slope,min_concentration,max_concentration,slope_adjusted,sign,bounded_ec50
0,EOS100028,0.0125,-7.986852,-9.834174,-6.139531,101.686898,-11.652444,0.50736,0.874958,0.0125,50.0,-1.0,=,0.50736
1,EOS100028,0.0500,1.652350,1.038550,2.266150,101.686898,-11.652444,0.50736,0.874958,0.0125,50.0,-1.0,=,0.50736
2,EOS100028,0.1625,20.600380,16.424481,24.776279,101.686898,-11.652444,0.50736,0.874958,0.0125,50.0,-1.0,=,0.50736
3,EOS100028,0.5000,43.084218,42.869863,43.298573,101.686898,-11.652444,0.50736,0.874958,0.0125,50.0,-1.0,=,0.50736
4,EOS100028,1.2500,65.632200,62.459374,68.805026,101.686898,-11.652444,0.50736,0.874958,0.0125,50.0,-1.0,=,0.50736


In [17]:
MAX_MIN_VALUE_THRESHOLD = 75
MIN_MAX_VALUE_THRESHOLD = 30

class ActivationDFEntry(FitDFEntry):
    min_value: float
    max_value: float
    mean_value: float
    sign: str
    operator: str
    top: float
    bottom: float
    final_slope: float

# why bound value with concentration?
# + it is not used in activity determination
def resolve_bounded_value(row: ActivationDFEntry) -> float:
    if row.min_value > MAX_MIN_VALUE_THRESHOLD:
        return row.min_concentration
    if row.max_value < MIN_MAX_VALUE_THRESHOLD:
        return row.max_concentration
    return row.mean_value

activation_df["value_bounded"] = activation_df.apply(resolve_bounded_value, axis=1)

In [18]:
def resolve_operator(row: ActivationDFEntry) -> str:
    if row.min_value > MAX_MIN_VALUE_THRESHOLD:
        return "<"
    if row.max_value < MIN_MAX_VALUE_THRESHOLD:
        return ">"
    return row.sign

activation_df["operator"] = activation_df.apply(resolve_operator, axis=1)

In [27]:
# un-adjusted slope used

activation_df["is_reverse_dose"] = activation_df.slope < 0

In [20]:
ACTIVITY_THRESHOLD = 10

activation_df["is_active"] = activation_df.value_bounded < ACTIVITY_THRESHOLD

In [21]:
def resolve_bottom(row: ActivationDFEntry) -> float:
    if row.operator == "=":
        return row.lower_limit
    return np.nan

activation_df["bottom"] = activation_df.apply(resolve_bottom, axis=1)

In [22]:
def resolve_top(row: ActivationDFEntry) -> float:
    if row.operator == "=":
        return row.upper_limit
    return np.nan

activation_df["top"] = activation_df.apply(resolve_top, axis=1)

In [23]:
def resolve_final_slope(row: ActivationDFEntry) -> float:
    if row.operator == "=":
        return row.slope
    return np.nan

activation_df["final_slope"] = activation_df.apply(resolve_final_slope, axis=1)

In [24]:
# what if unknown?

def resolve_activity_final(row: ActivationDFEntry) -> str:
    if row.ec50 >= 10 or row.top <= 30:
        return "inactive"
    if row.top > 60:
        return "active"
    if row.top > 30:
        return "inconclusive"
    return "unknown"


activation_df["activity_final"] = activation_df.apply(resolve_activity_final, axis=1)
    

In [25]:
activation_df["is_partially_active"] = (activation_df.top > 30) & (activation_df.top < 80) & (activation_df.ec50 < 10)

In [26]:
activation_df.to_excel("active_prototype.xlsx")